In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
%matplotlib inline

import numpy as np
import scipy.io as io
from pyDOE import lhs
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms

from complexPyTorch.complexLayers import ComplexLinear

import cplxmodule
from cplxmodule import cplx
from cplxmodule.nn import RealToCplx, CplxToReal, CplxSequential, CplxToCplx
from cplxmodule.nn import CplxLinear, CplxModReLU, CplxAdaptiveModReLU, CplxModulus, CplxAngle

# To access the contents of the parent dir
import sys; sys.path.insert(0, '../')
import os
from scipy.io import loadmat
from utils import *
from models import TorchComplexMLP, ImaginaryDimensionAdder, cplx2tensor, ComplexTorchMLP, complex_mse
from preprocess import *

# Model selection
from sparsereg.model import STRidge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge
from pde_diff import TrainSTRidge, FiniteDiff, print_pde
from RegscorePy.bic import bic

# Fancy optimizers
from madgrad import MADGRAD
from lbfgsnew import LBFGSNew

/usr/local/lib/python3.9/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.linear_model.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)


#### Data preparation

In [2]:
data = loadmat('../deephpms_data/KS.mat')

t = data['t'].flatten()[:,None]
x = data['x'].flatten()[:,None]
Exact = np.real(data['usol'])

X_sol, T_sol = np.meshgrid(x, t)

x_star = X_sol.flatten()[:,None]
t_star = T_sol.flatten()[:,None]

X_star = np.hstack((x_star, t_star))
u_star = Exact.T.flatten()[:,None]

# DATA_PATH = '../PDE_FIND_experimental_datasets/kuramoto_sivishinky.mat'
# X, T, Exact = space_time_grid(data_path=DATA_PATH, real_solution=True, uniform=True, x_limit=None, t_limit=None)
# X_star, u_star = get_trainable_data(X, T, Exact)

# Bound
ub = X_star.max(axis=0)
lb = X_star.min(axis=0)

# For identification
N = 10000
# idx = np.arange(N)
idx = np.random.choice(X_star.shape[0], N, replace=False)
X_train = X_star[idx,:]
u_train = u_star[idx,:]

print("Training with", N, "samples")

# Unsup data
include_N_res = False
if include_N_res:
    N_res = N//2
    idx_res = np.array(range(X_star.shape[0]-1))[~idx]
    idx_res = np.random.choice(idx_res.shape[0], N_res, replace=True)
    X_res = X_star[idx_res, :]
    print(f"Training with {N_res} unsup samples")
    X_u_train = np.vstack([X_train, X_res])
    u_train = np.vstack([u_train, torch.rand(X_res.shape[0], 1) - 1000])
    # del X_res
else: print("Not including N_res")
    
# Convert to torch.tensor
X_train = to_tensor(X_train, True)
u_train = to_tensor(u_train, False)
X_star = to_tensor(X_star, True)
u_star = to_tensor(u_star, False)

# lb and ub are used in adversarial training
scaling_factor = 1.0
lb = scaling_factor*to_tensor(lb, False)
ub = scaling_factor*to_tensor(ub, False)

# Feature names
feature_names=('uf', 'u_x', 'u_xx', 'u_xxx', 'u_xxxx')

Training with 10000 samples
Not including N_res


In [3]:
class Network(nn.Module):
    def __init__(self, model, index2features=None, scale=False, lb=None, ub=None):
        super(Network, self).__init__()
        # pls init the self.model before
        self.model = model
        # For tracking, the default tup is for the burgers' equation.
        self.index2features = index2features
        print("Considering", self.index2features)
        self.diff_flag = diff_flag(self.index2features)
        self.uf = None
        self.scale = scale
        self.lb, self.ub = lb, ub
        
    def xavier_init(self, m):
        if type(m) == nn.Linear:
            torch.nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(0.01)

    def forward(self, x, t):
        if not self.scale: self.uf = self.model(torch.cat([x, t], dim=1))
        else: self.uf = self.model(self.neural_net_scale(torch.cat([x, t], dim=1)))
        return self.uf
    
    def get_selector_data(self, x, t):
        uf = self.forward(x, t)
        u_t = self.gradients(uf, t)[0]
        
        ### PDE Loss calculation ###
        # Without calling grad
        derivatives = []
        for t in self.diff_flag[0]:
            if t=='uf': derivatives.append(uf)
            elif t=='x': derivatives.append(x)
        # With calling grad
        for t in self.diff_flag[1]:
            out = uf
            for c in t:
                if c=='x': out = self.gradients(out, x)[0]
                elif c=='t': out = self.gradients(out, t)[0]
            derivatives.append(out)
        
        return torch.cat(derivatives, dim=1), u_t
    
    def gradients(self, func, x):
        return grad(func, x, create_graph=True, retain_graph=True, grad_outputs=torch.ones(func.shape))
    
    def neural_net_scale(self, inp):
        return 2*(inp-self.lb)/(self.ub-self.lb)-1

In [4]:
class AttentionSelectorNetwork(nn.Module):
    def __init__(self, layers, prob_activation=torch.sigmoid, bn=None, reg_intensity=0.1):
        super(AttentionSelectorNetwork, self).__init__()
        # Nonlinear model, Training with PDE reg.
        assert len(layers) > 1
        self.linear1 = nn.Linear(layers[0], layers[0])
        self.prob_activation = prob_activation
        self.nonlinear_model = TorchMLP(dimensions=layers, activation_function=nn.Tanh, bn=bn, dropout=nn.Dropout(p=0.1))
        self.latest_weighted_features = None
        self.th = 0.1
        self.reg_intensity = reg_intensity
        
    def xavier_init(self, m):
        if type(m) == nn.Linear:
            torch.nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(0.01)
        
    def forward(self, inn):
        return self.nonlinear_model(inn*self.weighted_features(inn))
    
    def weighted_features(self, inn):
        self.latest_weighted_features = self.prob_activation(self.linear1(inn)).mean(axis=0)
        return self.latest_weighted_features
    
    def loss(self, X_input, y_input):
        ut_approx = self.forward(X_input)
        mse_loss = F.mse_loss(ut_approx, y_input, reduction='mean')
        reg_term = F.relu(self.latest_weighted_features-self.th)
        return mse_loss+self.reg_intensity*(torch.norm(reg_term, p=0)+(torch.tensor([1.0, 1.0, 2.0, 3.0, 4.0])*reg_term).sum())

class SemiSupModel(nn.Module):
    def __init__(self, network, selector, normalize_derivative_features=False, mini=None, maxi=None):
        super(SemiSupModel, self).__init__()
        self.network = network
        self.selector = selector
        self.normalize_derivative_features = normalize_derivative_features
        self.mini = mini
        self.maxi = maxi
        
    def forward(self, X_u_train):
        X_selector, y_selector = self.network.get_selector_data(*dimension_slicing(X_u_train))
        if self.normalize_derivative_features:
            X_selector = (X_selector-self.mini)/(self.maxi-self.mini)
        unsup_loss = self.selector.loss(X_selector, y_selector)
        return self.network.uf, unsup_loss

In [5]:
### Version with normalized derivatives ###
# pretrained_state_dict = cpu_load("../inverse_KS/saved_path_inverse_ks/semisup_model_with_LayerNormDropout_without_physical_reg_trained30000labeledsamples_trained0unlabeledsamples.pth")
# pretrained_state_dict = cpu_load("semisup_model_state_dict.pth")
pretrained_state_dict = None
use_pretrained_weights = False
lets_pretrain = True

semisup_model = SemiSupModel(network=Network(
                                    model=TorchMLP(dimensions=[2, 50, 50, 50 ,50, 50, 1],
                                                   activation_function=nn.Tanh,
                                                   bn=nn.LayerNorm, dropout=None),
                                    index2features=feature_names, scale=True, lb=lb, ub=ub),
                            selector=AttentionSelectorNetwork([len(feature_names), 50, 50, 1], prob_activation=nn.Softmax(dim=1), bn=nn.LayerNorm),
                            normalize_derivative_features=True,
                            mini=None,
                            maxi=None)

if use_pretrained_weights:
    print("Use pretrained weights")
    semisup_model.load_state_dict(pretrained_state_dict)
    referenced_derivatives, u_t = semisup_model.network.get_selector_data(*dimension_slicing(X_train))
    semisup_model.mini = torch.min(referenced_derivatives, axis=0)[0].detach().requires_grad_(False)
    semisup_model.maxi = torch.max(referenced_derivatives, axis=0)[0].detach().requires_grad_(False)
#     semisup_model.mini = tmp.min(axis=0)[0].requires_grad_(False)
#     semisup_model.maxi = tmp.max(axis=0)[0].requires_grad_(False)

Using old implementation of TorchMLP. See models.py for more new model-related source code.
Considering ('uf', 'u_x', 'u_xx', 'u_xxx', 'u_xxxx')
Using old implementation of TorchMLP. See models.py for more new model-related source code.


In [6]:
if lets_pretrain:
    print("Pretraining")
    pretraining_optimizer = LBFGSNew(semisup_model.network.parameters(),
                                     lr=1e-1, max_iter=300,
                                     max_eval=int(300*1.25), history_size=150,
                                     line_search_fn=True, batch_mode=False)

    semisup_model.network.train()    
    for i in range(120):
        def pretraining_closure():
            global N, X_u_train, u_train
            if torch.is_grad_enabled():
                pretraining_optimizer.zero_grad()
            # Only focusing on first [:N, :] elements
            mse_loss = F.mse_loss(semisup_model.network(*dimension_slicing(X_train[:N, :])), u_train[:N, :])
            if mse_loss.requires_grad:
                mse_loss.backward(retain_graph=False)
            return mse_loss

        pretraining_optimizer.step(pretraining_closure)
            
        if (i%10)==0:
            l = pretraining_closure()
            curr_loss = l.item()
            print("Epoch {}: ".format(i), curr_loss)

            # Sneak on the test performance...
            semisup_model.network.eval()
            test_performance = F.mse_loss(semisup_model.network(*dimension_slicing(X_star)).detach(), u_star).item()
            string_test_performance = scientific2string(test_performance)
            print('Test MSE:', string_test_performance)
    
#     if best_state_dict is not None: semisup_model.load_state_dict(best_state_dict)
    print("Computing derivatives features")
    semisup_model.eval()
    referenced_derivatives, u_t = semisup_model.network.get_selector_data(*dimension_slicing(X_train))
    semisup_model.mini = torch.min(referenced_derivatives, axis=0)[0].detach().requires_grad_(False)
    semisup_model.maxi = torch.max(referenced_derivatives, axis=0)[0].detach().requires_grad_(False)

#     semisup_model.mini = tmp.min(axis=0)[0].requires_grad_(False)
#     semisup_model.maxi = tmp.max(axis=0)[0].requires_grad_(False)

Pretraining
Epoch 0:  0.003545314073562622
Test MSE: 3.8e-03
Epoch 10:  2.7006650270777754e-05
Test MSE: 3.0e-05
Epoch 20:  1.9923425497836433e-05
Test MSE: 2.3e-05
Epoch 30:  1.8414441001368687e-05
Test MSE: 2.1e-05
Epoch 40:  1.579972740728408e-05
Test MSE: 1.8e-05
Epoch 50:  1.4198953067534603e-05
Test MSE: 1.6e-05
Epoch 60:  1.3893720279156696e-05
Test MSE: 1.6e-05
Epoch 70:  1.3472734281094745e-05
Test MSE: 1.5e-05
Epoch 80:  1.31282895381446e-05
Test MSE: 1.5e-05
Epoch 90:  1.2774522474501282e-05
Test MSE: 1.5e-05
Epoch 100:  1.2243061064509675e-05
Test MSE: 1.4e-05
Epoch 110:  1.1576370525290258e-05
Test MSE: 1.3e-05
Computing derivatives features


In [16]:
n_test = 10000
idx_test = np.arange(n_test)
referenced_derivatives, u_t = semisup_model.network.get_selector_data(*dimension_slicing(X_star[idx_test, :]))

In [17]:
print((((u_t+referenced_derivatives[:, 4:5]+(referenced_derivatives[:, 0:1]*referenced_derivatives[:, 1:2])+referenced_derivatives[:, 2:3]))**2).mean())
referenced_derivatives = to_numpy(referenced_derivatives); u_t = to_numpy(u_t)

alpha = 1
const_range = (-1.5, 1.5)

X_input = referenced_derivatives
y_input = u_t

poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=True)
X_input = poly.fit_transform(X_input)

poly_feature_names = poly.get_feature_names(feature_names)
for i, f in enumerate(poly_feature_names):
    poly_feature_names[i] = f.replace(" ", "*")

tensor(0.0050, grad_fn=<MeanBackward0>)


In [22]:
w = TrainSTRidge(X_input[:, :10], y_input, 1e-10, 1000, maxit=1000, STR_iters=10, split=0.8, l0_penalty=1)
print("PDE derived using STRidge")
print_pde(w, poly_feature_names[:10])

PDE derived using STRidge
u_t = (-0.840458 +0.000000i)u_xx
    + (-0.741539 +0.000000i)u_xxxx
    + (-0.790747 +0.000000i)uf*u_x
   


In [23]:
fdc = FinDiffCalculator(X_sol, T_sol, Exact.T, acc_order=10)

# Even if you use the true vals of dx and dt -> The finite difference is not working with a non-uniform grid
# fdc.dx = 0.09817477042468103
# fdc.dt = 0.4

fd_u_t = to_tensor(fdc.finite_diff(1, diff_order=1), False)
fd_derivatives = fdc.finite_diff_from_feature_names(feature_names)
for d in fd_derivatives: fd_derivatives[d] = to_tensor(fd_derivatives[d], False)

Do not use this class with complex-valued input arrays.
This class applies 1 transpose to the Exact before doing the job.
dx = 0.039138943248532954
dt = 0.2


In [24]:
tmp = None
for f in feature_names:
    print('Concatenating', f)
    if tmp == None: tmp = fd_derivatives[f]
    else: tmp = torch.cat([tmp, fd_derivatives[f]], dim=-1)

fd_derivatives = tmp[:, :]
(((fd_u_t+fd_derivatives[:, 4:5]+(fd_derivatives[:, 0:1]*fd_derivatives[:, 1:2])+fd_derivatives[:, 2:3]))**2).mean().item()

Concatenating uf
Concatenating u_x
Concatenating u_xx
Concatenating u_xxx
Concatenating u_xxxx


7.749761425657198e-05